In [15]:
import os

In [16]:
%pwd
os.system('C:')
os.chdir(r'C:\Users\LENOVO\Desktop\PROJECT\Chest-Cancer-Classification-using-MLflow-DVC')


In [17]:
%pwd

'C:\\Users\\LENOVO\\Desktop\\PROJECT\\Chest-Cancer-Classification-using-MLflow-DVC'

In [18]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list


In [19]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
from src.cnnClassifier.config.configuration import ConfigurationManager
import tensorflow as tf
import os

class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Data")
        create_directories([Path(training.root_dir)])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )
        return training_config


In [28]:
import os
import tensorflow as tf
from pathlib import Path

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        # Load the pre-trained model
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)
        
        # Modify the output layer to 4 classes
        x = self.model.output
        output_layer = tf.keras.layers.Dense(3, activation='softmax',name='final_output')(x)
        self.model = tf.keras.models.Model(inputs=self.model.input, outputs=output_layer)

    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            class_mode='categorical',
            interpolation="bilinear"
        )

        # Training generator
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.model.compile(
            optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [30]:
try:
    CONFIG_FILE_PATH = Path(r'C:/Users/LENOVO/Desktop/PROJECT/Chest-Cancer-Classification-using-MLflow-DVC/config/config.yaml')
    PARAMS_FILE_PATH = Path(r'C:/Users/LENOVO/Desktop/PROJECT/Chest-Cancer-Classification-using-MLflow-DVC/params.yaml')

    config = ConfigurationManager(config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH)
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
except Exception as e:
    raise e


[2025-03-08 16:12:42,161: INFO: common: yaml file: C:\Users\LENOVO\Desktop\PROJECT\Chest-Cancer-Classification-using-MLflow-DVC\config\config.yaml loaded successfully]


[2025-03-08 16:12:42,247: INFO: common: yaml file: C:\Users\LENOVO\Desktop\PROJECT\Chest-Cancer-Classification-using-MLflow-DVC\params.yaml loaded successfully]
[2025-03-08 16:12:42,255: INFO: common: created directory at: artifacts]
[2025-03-08 16:12:42,258: INFO: common: created directory at: artifacts\training]
Found 199 images belonging to 3 classes.
Found 801 images belonging to 3 classes.
Epoch 1/6
25/25 [==============================] - 148s 6s/step - loss: 1.1065 - accuracy: 0.5865 - val_loss: 1.0774 - val_accuracy: 0.6354
Epoch 2/6
25/25 [==============================] - 144s 6s/step - loss: 1.0725 - accuracy: 0.6151 - val_loss: 1.0588 - val_accuracy: 0.6354
Epoch 3/6
25/25 [==============================] - 144s 6s/step - loss: 1.0736 - accuracy: 0.6047 - val_loss: 1.0416 - val_accuracy: 0.6354
Epoch 4/6
25/25 [==============================] - 144s 6s/step - loss: 1.0486 - accuracy: 0.6112 - val_loss: 1.0262 - val_accuracy: 0.6354
Epoch 5/6
25/25 [=========================